<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Model Application

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the labels to the catalog.  The ROC curve is also calculated and uploaded.


In [1]:
# Prerequisites to configure colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install deriva
    !pip install bdbag
    !pip install --upgrade --force pydantic
    !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
    !pip install setuptools_git_versioning


In [2]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [3]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [4]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-07-01 14:42:27,615 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-07-01 14:42:27,616 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [5]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-C9AP"      # Configuration file for this run.  Needs to be changed for each execution.

In [6]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-07-01 14:42:32,752 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-07-01 14:42:32,753 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [7]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()


2024-07-01 14:42:34,892 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_prediction_more_metrics_sreenidhi_june_24_2024.json] transfer successful. 0.87 KB transferred. Elapsed time: 0:00:00.000084.
2024-07-01 14:42:34,892 - INFO - Verifying MD5 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_prediction_more_metrics_sreenidhi_june_24_2024.json]
2024-07-01 14:42:34,909 - INFO - Configuration validation successful!
2024-07-01 14:42:40,218 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024.h5] transfer successful. 232.71 MB transferred at 72.87 MB/second. Elapsed time: 0:00:03.193504.
2024-07-01 14:42:40,422 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json] transfer successful. 0.69 KB tra

{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/sreenidhi/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'VGG19_Catalog_Model_Prediction',
    'rid': '2-C9AT'}],
  'Execution_Asset_Type': [{'name': 'VGG19_Catalog_Model_Prediction',
    'rid': '2-C9AW'}]},
 'execution_rid': '2-C9B6',
 'workflow_rid': '2-C9AY',
 'bag_paths': [PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')],
 'assets_paths': [PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024.h5'),
  PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json')],
 'configuration_path': PosixPath('/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_prediction_more_metrics_sreenidhi_june_24_2024.json')}

Algorithm was trained on cropped images, so take the raw images and bounding boxes and apply, storing the results in the working directory.

In [8]:
configuration_records

ConfigurationRecord(caching_dir=PosixPath('/data'), working_dir=PosixPath('/data/sreenidhi/EyeAI_working'), vocabs={'Workflow_Type': [Term(name='VGG19_Catalog_Model_Prediction', rid='2-C9AT')], 'Execution_Asset_Type': [Term(name='VGG19_Catalog_Model_Prediction', rid='2-C9AW')]}, execution_rid='2-C9B6', workflow_rid='2-C9AY', bag_paths=[PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')], assets_paths=[PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024.h5'), PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json')], configuration_path=PosixPath('/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_prediction_more_metrics_sreenidhi_june_24_2024.json'))

In [9]:
str(EA.working_dir)

'/data/sreenidhi/EyeAI_working'

In [10]:
# @title Get Cropped Images
cropped_image_path, cropped_csv = EA.create_cropped_images(str(configuration_records.bag_paths[0]),
                                                           output_dir = str(EA.working_dir),
                                                           crop_to_eye=True)

In [16]:
import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

def analyze_graded_test_dataset(base_path):
    main_folders = ['Image_cropped'] #, 
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nAnalyzing {main_folder} folder:")
        
        total_files = 0
        for angle_folder in os.listdir(main_folder_path):
            angle_folder_path = os.path.join(main_folder_path, angle_folder)
            if os.path.isdir(angle_folder_path):
                file_count = count_files(angle_folder_path)
                print(f"  {angle_folder}: {file_count} images")
                total_files += file_count
        
        print(f"Total images in {main_folder}: {total_files}")

# Usage
base_path = "/data/sreenidhi/EyeAI_working/"
analyze_graded_test_dataset(base_path)



Analyzing Image_cropped folder:
  2SKC_No_Glaucoma: 526 images
  2SKA_Suspected_Glaucoma: 568 images
Total images in Image_cropped: 1094


In [11]:
best_hyper_parameters_json_path = str(configuration_records.assets_paths[1])
best_hyper_parameters_json_path

'/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'

In [13]:
model_path = configuration_records.assets_paths[0]
model_path 

PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024.h5')

In [14]:
cropped_image_path

PosixPath('/data/sreenidhi/EyeAI_working/Image_cropped')

In [18]:

output_path = str(EA.working_dir) + "/Execution_Assets/" + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)

output_path

'/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model_Prediction'

Import the actual model code and then run against the input dataset specified in the configuration file.  

In [22]:
# @title Execute Proecss algorithm (Test model)
from eye_ai.models.vgg19_diagnosis_predict_more_metrics import prediction

with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  prediction(
      model_path = model_path, 
      cropped_image_path = cropped_image_path, 
      output_dir = output_path, 
      best_hyperparameters_json_path = best_hyper_parameters_json_path)

Found 1094 images belonging to 2 classes.
35/35 [==============================] - 6s 163ms/step


2024-07-01 14:55:21,005 - INFO - 
Metrics for VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024:
2024-07-01 14:55:21,006 - INFO - Accuracy: 0.8080438756855576
2024-07-01 14:55:21,006 - INFO - Precision: 0.8653061224489796
2024-07-01 14:55:21,007 - INFO - Recall: 0.7464788732394366
2024-07-01 14:55:21,007 - INFO - F1 Score: 0.8015122873345936
2024-07-01 14:55:21,008 - INFO - ROC AUC: 0.8930976543672682
2024-07-01 14:55:21,022 - INFO - Classification Report for VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024:
2024-07-01 14:55:21,022 - INFO - {
  "2SKC_No_Glaucoma": {
    "precision": 0.7615894039735099,
    "recall": 0.8745247148288974,
    "f1-score": 0.8141592920353983,
    "support": 526.0
  },
  "2SKA_Suspected_Glaucoma": {
    "precision": 0.8653061224489796,
    "recall": 0.7464788732394366,
    "f1-score": 0.8015122873345936,
    "support": 568.0
  },
  "accuracy": 0.8080438756855576,
  "macro 


Metrics for VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024:
Accuracy: 0.8080438756855576
Precision: 0.8653061224489796
Recall: 0.7464788732394366
F1 Score: 0.8015122873345936
ROC AUC: 0.8930976543672682

Classification Report for VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024:
                         precision    recall  f1-score   support

       2SKC_No_Glaucoma       0.76      0.87      0.81       526
2SKA_Suspected_Glaucoma       0.87      0.75      0.80       568

               accuracy                           0.81      1094
              macro avg       0.81      0.81      0.81      1094
           weighted avg       0.82      0.81      0.81      1094



2024-07-01 14:55:21,828 - INFO - Confusion matrix saved as VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024_confusion_matrix.png (300 DPI)
2024-07-01 14:55:22,276 - INFO - ROC curve saved as VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024_roc_curve.png (300 DPI)
2024-07-01 14:55:22,280 - INFO - Data saved to VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024_predictions_results.csv
2024-07-01 14:55:22,281 - INFO - Metrics saved to VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024_metrics.json


In [24]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, True)


2024-07-01 14:56:25,032 - INFO - Initializing uploader: GenericUploader v1.7.1 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-07-01 14:56:25,033 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-07-01 14:56:25,034 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-07-01 14:56:25,070 - INFO - Checking for updated configuration...
2024-07-01 14:56:25,203 - INFO - Updated configuration found.
2024-07-01 14:56:25,205 - INFO - Scanning files in directory [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model_Prediction]...
2024-07-01 14:56:25,209 - INFO - Including file: [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model_Prediction/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024_con